In [21]:
from pathlib import Path

estimator_path = '/Users/vobornij/projects/polymarket/data/estimator/2025-08-24/bitcoin-up-or-down-august-24-9am-et_0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35.jsonl'

market = Path(estimator_path).name.rsplit('_', 1)[-1].removesuffix('.jsonl')
slug = Path(estimator_path).name.rsplit('_', 1)[0]
print(f'market: {market} slug: {slug}')

market: 0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35 slug: bitcoin-up-or-down-august-24-9am-et


In [22]:
import pathlib, json, time, datetime, requests
from typing import List, Dict, Any, Optional

MARKET_ID = market

BASE_URL = "https://data-api.polymarket.com/trades"  # per docs
MAX_LIMIT = 500  # API max
OUTPUT_DIR = pathlib.Path('/Users/vobornij/projects/polymarket/data/trades')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def fetch_trades_batch(market: str, limit: int = MAX_LIMIT, offset: int = 0, taker_only: bool = True) -> List[Dict[str, Any]]:
    params = {
        'market': market,
        'limit': min(limit, MAX_LIMIT),
        'offset': offset,
        'takerOnly': str(taker_only).lower(),  # 'true' / 'false'
    }
    r = requests.get(BASE_URL, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list):
        raise ValueError(f'Unexpected response type: {type(data)} | {data}')
    return data

base = f"{market[:16]}"
jsonl_path = OUTPUT_DIR / f"{base}.jsonl"

def download_all_trades(market: str, batch_limit: int = MAX_LIMIT, sleep: float = 0.1, max_batches: Optional[int] = None, taker_only: bool = True) -> pathlib.Path:
    total = 0
    offset = 0
    batch_no = 0
    with jsonl_path.open('w') as f:
        while True:
            batch_no += 1
            batch = fetch_trades_batch(market, limit=batch_limit, offset=offset, taker_only=taker_only)
            if not batch:
                break
            for tr in batch:
                f.write(json.dumps(tr) + '\n')
            got = len(batch)
            total += got
            offset += got
            # Stop conditions
            if got < batch_limit:
                # Last page
                break
            if max_batches and batch_no >= max_batches:
                break
            if sleep:
                time.sleep(sleep)
    summary = {
        'market': market,
        'file': jsonl_path.name,
        'total_trades': total,
        'batches': batch_no,
        'note': 'Trades returned newest-first (descending timestamp). Offset pagination accumulates all until exhaustion.'
    }
    (OUTPUT_DIR / f"{base}_summary.json").write_text(json.dumps(summary, indent=2))
    print(f"Saved {total} trades across {batch_no} batch(es) -> {jsonl_path}")
    return jsonl_path


# download trades if file does not exist: 
if not jsonl_path.exists():
    download_all_trades(MARKET_ID)


In [23]:
# Load latest downloaded trades for the current `market` into a DataFrame (asset forced to string)
import pandas as pd, pathlib, json, datetime, os
from typing import List, Dict, Any

MARKET_ID = globals().get('market')
if not MARKET_ID:
    raise ValueError("`market` variable not defined. Set it before running this cell.")

trades_dir = pathlib.Path('/Users/vobornij/projects/polymarket/data/trades')
prefix = MARKET_ID[:16] + '_'


print(f'Loading trades from {jsonl_path}')

# Manually parse JSONL to control dtypes and preserve very large identifiers as strings
rows: List[Dict[str, Any]] = []
with jsonl_path.open() as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        # Force asset to string exactly as in file (avoid pandas numeric inference / float formatting)
        if 'asset' in obj:
            obj['asset'] = str(obj['asset'])
        rows.append(obj)

df = pd.DataFrame(rows)

# # Ensure pandas didn't coerce asset
# if 'asset' in df.columns:
#     df['asset'] = df['asset'].astype('string')  # pandas string dtype retains exact text

# Add a human-readable UTC datetime column if timestamp present (assumes seconds)
if 'timestamp' in df.columns:
    def _to_dt(x):
        try:
            return datetime.datetime.fromtimestamp(int(x), datetime.timezone.utc)
        except Exception:
            return pd.NaT
    df['dt'] = df['timestamp'].map(_to_dt)

# Sort ascending by timestamp if present
if 'timestamp' in df.columns:
    df.sort_values('timestamp', inplace=True)

print(f'Trades loaded: {len(df)} rows')
if 'asset' in df.columns:
    sample_asset = df['asset'].iloc[0]
    print(f'asset dtype: {df['asset'].dtype}; sample asset length: {len(sample_asset)}')

df.head(5)

Loading trades from /Users/vobornij/projects/polymarket/data/trades/0x0e273c0dec081f.jsonl
Trades loaded: 1690 rows
asset dtype: object; sample asset length: 77


,proxyWallet,side,asset,conditionId,size,price,timestamp,title,slug,icon,eventSlug,outcome,outcomeIndex,name,pseudonym,bio,profileImage,profileImageOptimized,transactionHash,dt
1689,0xe33d60a1aa150ae45bad73fbe9538e9ed1c86cd1,BUY,24877577346649146396079163544060191733965845783529843138970254568033322885572,0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35,5.882351,0.51,1755963671,"Bitcoin Up or Down - August 24, 9AM ET",bitcoin-up-or-down-august-24-9am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-august-24-9am-et,Up,0,gogi1983,Low-Simvastatin,,,,0x49619d108b54f843e58cb980067982f84e596716f74c98f1983af88697770be6,2025-08-23 15:41:11+00:00
1688,0x09bc1527a6f9ebc0fcf0c5b555dfdf8ad56a0082,BUY,24877577346649146396079163544060191733965845783529843138970254568033322885572,0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35,1.960783,0.51,1756027311,"Bitcoin Up or Down - August 24, 9AM ET",bitcoin-up-or-down-august-24-9am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-august-24-9am-et,Up,0,bobofish,Happy-Go-Lucky-Disclaimer,,https://polymarket-upload.s3.us-east-2.amazonaws.com/profile-image-3263295-c9b704d8-c868-4934-a99b-22b387ab355c.jpeg,,0xf719e6e8e326577fa331aa9b113842902af9f491cad4d054d4ff6b7b5dcee206,2025-08-24 09:21:51+00:00
1687,0xb9fc8078fd6c0275c631ec10fcf8d5cc52d6da76,BUY,24877577346649146396079163544060191733965845783529843138970254568033322885572,0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35,100.000000,0.51,1756032243,"Bitcoin Up or Down - August 24, 9AM ET",bitcoin-up-or-down-august-24-9am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-august-24-9am-et,Up,0,jacobfox,Mature-Sunbeam,,,,0xa0ae9d4e77a409af14da0dd45ad9ac9fd965d72e365fb855e33673c68e0e382a,2025-08-24 10:44:03+00:00
1686,0x196975df44758121f3f395c6af9d97018750543a,BUY,24877577346649146396079163544060191733965845783529843138970254568033322885572,0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35,5.000000,0.51,1756032255,"Bitcoin Up or Down - August 24, 9AM ET",bitcoin-up-or-down-august-24-9am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-august-24-9am-et,Up,0,ffty,Glorious-Deep,,,,0x98b0357c014019c0842c1b8ddae088b48f4f11ed706fc6fcc8924d98d03987c8,2025-08-24 10:44:15+00:00
1685,0x0ea266c80ed008bf81a221a10ffca725da6979db,BUY,24877577346649146396079163544060191733965845783529843138970254568033322885572,0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35,9.803920,0.51,1756035229,"Bitcoin Up or Down - August 24, 9AM ET",bitcoin-up-or-down-august-24-9am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-august-24-9am-et,Up,0,DenisRB,Alert-Archrival,,https://polymarket-upload.s3.us-east-2.amazonaws.com/profile-image-3170445-2006bdd9-32bc-49a7-ab9a-8876aa62f08a.jpg,,0x870b0f7db156ebbc695c3280f03ae476d08581525990f07fba5446476f29d321,2025-08-24 11:33:49+00:00


In [24]:
# Compute per-wallet UP positions at a cutoff timestamp
from typing import Optional
import pandas as pd, math, datetime

UP_LABELS = {"Yes", "YES", "Up", "UP"}
DOWN_LABELS = {"No", "NO", "Down", "DOWN"}

if 'dt' not in df.columns:
    raise ValueError("DataFrame df must contain 'dt' datetime column; re-run load cell.")

required_cols = {'proxyWallet', 'side', 'size', 'price', 'name'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required trade columns: {missing}")

if 'outcome' not in df.columns and 'outcomeIndex' not in df.columns:
    raise ValueError("Need either 'outcome' or 'outcomeIndex' column to classify trades.")

work = df.copy()

def classify_outcome(row):
    if 'outcome' in row and isinstance(row['outcome'], str):
        o = row['outcome']
        if o in UP_LABELS:
            return 'UP'
        if o in DOWN_LABELS:
            return 'DOWN'

work['__class'] = work.apply(classify_outcome, axis=1)
unknown_ct = (work['__class'] == 'UNKNOWN').sum()
if unknown_ct:
    print(f"Warning: {unknown_ct} trades could not be classified (tagged UNKNOWN). They are ignored.")
work = work[work['__class'] != 'UNKNOWN']

def compute_deltas(row):
    side = row['side']
    size = float(row['size'])
    price = float(row['price'])
    cls = row['__class']
    if cls == 'UP':
        if side == 'BUY':
            return size, -price * size
        else:
            return -size, +price * size
    else:  # DOWN
        price_up = 1.0 - price
        if side == 'BUY':
            return -size, +price_up * size
        else:
            return +size, -price_up * size

work[['delta_shares_up','cash_flow']] = work.apply(lambda r: pd.Series(compute_deltas(r)), axis=1)

agg = work.groupby('proxyWallet', as_index=False).agg(
    name=('name','first'),
    shares_up=('delta_shares_up','sum'),
    cash_flow=('cash_flow','sum'),
    trade_count=('delta_shares_up','count')
)

def avg_entry(row):
    if row['shares_up'] > 0:
        return (-row['cash_flow']) / row['shares_up'] if row['shares_up'] != 0 else math.nan
    return math.nan

agg['avg_entry_price_est'] = agg.apply(avg_entry, axis=1)
agg.sort_values('shares_up', ascending=False, inplace=True)
agg.reset_index(drop=True, inplace=True)

agg.head(25)

,proxyWallet,name,shares_up,cash_flow,trade_count,avg_entry_price_est
0,0x9e9b318c915b289ea516fc1f46899ccf5e749e90,AminorNL,2655.616197,217.267026,6,-0.081814
1,0x3d2d66eb933cfa7aa7b9fc21e6614f080de99360,,1804.500000,-1117.235000,26,0.619138
2,0x8b424e123b219710a6ae219f6b61290879534eb6,MartinDupon,1763.450971,-898.999995,37,0.509796
3,0xb563eb0184543459596fd1011d013b7451600115,,1423.080000,-1288.803200,51,0.905644
4,0x80f8bb68361f2a2ff2709e59a544a98ab2946713,SergioVenezia,1145.507923,-599.999999,1,0.523785
5,0x118ed6e1e562ec54f1fa6e7f0a2741fe0d32ea4b,SellLowBuyHigh,1045.000000,-569.700000,3,0.545167
6,0x2edb696cea63b3d02f6400857403d38ce2f43780,8787dada,1014.583332,-474.999999,1,0.468172
7,0xcb8cf3a14bb4e679287b8d8ae5f0830a22920a55,badseer,925.297964,-1297.281570,16,1.402015
8,0x98b5ca5a6a02a75477cfb69e37ba97b8a70c4c9e,,710.000000,-472.000000,10,0.664789
9,0x1246b7914482f50530b5b1e0e14e442c4f5717bf,silverhandbtc,589.833750,-451.590000,1,0.765623


In [25]:
estimator_path

'/Users/vobornij/projects/polymarket/data/estimator/2025-08-24/bitcoin-up-or-down-august-24-9am-et_0x0e273c0dec081f45e34653d8f0a1bd8f7dd15d7ba23fb5d57fee55781d86ad35.jsonl'

In [30]:
# Plot btcPrice from estimator JSONL vs Polymarket UP price + model estimate (cropped to estimator time range)
import json, pathlib, pandas as pd, plotly.graph_objects as go, datetime, re
from typing import List, Dict, Any

# Ensure trades df exists
if 'df' not in globals():
    raise ValueError("Trades DataFrame `df` not found. Run the load trades cell first.")

plot_df = df.copy()
for col in ['price','size','dt']:
    if col not in plot_df.columns:
        raise ValueError(f'Missing column {col} in trades DataFrame.')

# Classify outcomes if needed
if '__class' not in plot_df.columns:
    UP_LABELS = {"Yes", "YES", "Up", "UP"}
    DOWN_LABELS = {"No", "NO", "Down", "DOWN"}
    def _cls(row):
        if 'outcome' in row and isinstance(row['outcome'], str):
            if row['outcome'] in UP_LABELS: return 'UP'
            if row['outcome'] in DOWN_LABELS: return 'DOWN'
        if 'outcomeIndex' in row:
            try:
                return 'UP' if int(row['outcomeIndex']) == 0 else 'DOWN'
            except Exception:
                return 'UNKNOWN'
        return 'UNKNOWN'
    plot_df['__class'] = plot_df.apply(_cls, axis=1)

plot_df = plot_df[plot_df['__class'] != 'UNKNOWN'].copy()
plot_df['up_price'] = plot_df.apply(lambda r: float(r['price']) if r['__class'] == 'UP' else 1.0 - float(r['price']), axis=1)
plot_df['size'] = plot_df['size'].astype(float)

# --- Load estimator file (provided path) ---
estimator_dir = pathlib.Path('/Users/vobornij/projects/polymarket/')
base_id = None
if 'market' in globals():
    base_id = re.sub(r'^0x','', str(market))

est_df = pd.DataFrame()
rows = []
with (estimator_dir / estimator_path).open() as f:
    for line in f:
        line=line.strip()
        if not line: continue
        try:
            rows.append(json.loads(line))
        except Exception:
            pass
if rows:
    est_df = pd.DataFrame(rows)
    # Timestamp
    tcol = 'timestamp' if 'timestamp' in est_df.columns else None
    if tcol is not None:
        est_df['dt'] = pd.to_datetime(est_df[tcol], utc=True, errors='coerce')
    else:
        for alt in ['time','ts','datetime','dt']:
            if alt in est_df.columns:
                est_df['dt'] = pd.to_datetime(est_df[alt], utc=True, errors='coerce')
                break
    est_df = est_df.dropna(subset=['dt'])
    # Numeric conversion for new format fields
    numeric_cols = ['btcPrice','target','estimate','seasonalSigma','sigmaEwmShrunkNormalized','sigmaEwmShrunkNormalizedFactor','backbone']
    for c in numeric_cols:
        if c in est_df.columns:
            est_df[c] = pd.to_numeric(est_df[c], errors='coerce')
    keep_cols = [c for c in ['dt','btcPrice','estimate','target','seasonalSigma','sigmaEwmShrunkNormalized','sigmaEwmShrunkNormalizedFactor','backbone'] if c in est_df.columns]
    est_df = est_df[keep_cols].sort_values('dt')

# Crop trades to estimator time range if estimator data present
if not est_df.empty:
    est_start, est_end = est_df['dt'].min(), est_df['dt'].max()
    plot_df = plot_df[(plot_df['dt'] >= est_start) & (plot_df['dt'] <= est_end)].copy()

# Recompute per-second weighted average after potential cropping
if not plot_df.empty:
    plot_df['dt_sec'] = plot_df['dt'].dt.floor('s')
    agg_up = (plot_df.groupby('dt_sec')
              .apply(lambda g: (g['up_price'] * g['size']).sum() / g['size'].sum(), include_groups=False)
              .rename('up_wap')
              .to_frame()
              .reset_index())
else:
    agg_up = pd.DataFrame(columns=['dt_sec','up_wap'])

In [27]:
est_df.iloc[0]

dt          2025-08-24 13:00:00+00:00
btcPrice                    114673.81
estimate                     0.499712
target                      114673.81
Name: 0, dtype: object

In [ ]:
# Updated plot for new estimator schema (no scalingFactor) + added volatility traces if present
# Primary y-axis now centered around target so target line aligns with 0.5 on secondary probability axis.
# Restored rich hover functionality with unified crosshair and per-trace custom hover templates.
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go

if 'est_df' not in globals() or est_df.empty:
    raise ValueError('est_df empty - run estimator load cell first.')

fig = make_subplots(specs=[[{"secondary_y": True}]])

hover_time_fmt = '%Y-%m-%d %H:%M:%S'

# BTC price
if 'btcPrice' in est_df.columns:
    fig.add_scatter(
        x=est_df['dt'], y=est_df['btcPrice'], name='BTC Price', secondary_y=False,
        line=dict(color='gray'),
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>BTC: %{y:.2f}<extra>BTC Price</extra>'
    )

# Target horizontal (record for centering)
if 'target' in est_df.columns and est_df['target'].notna().any():
    targ = est_df['target'].dropna().iloc[0]
    fig.add_scatter(
        x=est_df['dt'], y=[targ]*len(est_df), name='Target', secondary_y=False,
        line=dict(color='black', dash='dash'),
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Target: '+ '{:.2f}'.format(targ) +'<extra>Target</extra>'
    )
else:
    targ = None

# Probabilities: aggregated WAP & raw trades
if 'agg_up' in globals() and not agg_up.empty:
    fig.add_scatter(
        x=agg_up['dt_sec'], y=agg_up['up_wap'], name='UP WAP', mode='lines', line=dict(color='royalblue'), secondary_y=True,
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>UP WAP: %{y:.3f}<extra>UP WAP</extra>'
    )

if 'plot_df' in globals() and not plot_df.empty and 'up_price' in plot_df.columns:
    fig.add_scatter(
        x=plot_df['dt'], y=plot_df['up_price'], name='UP Trades', mode='markers',
        marker=dict(size=4, color='royalblue', opacity=0.3), secondary_y=True,
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>UP Trade: %{y:.3f}<br>Size: %{customdata[0]:.2f}<extra>Trade</extra>',
        customdata=np.stack([plot_df['size'].values], axis=-1)
    )

# Model estimate probability
if 'estimate' in est_df.columns:
    fig.add_scatter(
        x=est_df['dt'], y=est_df['estimate'], name='Model Estimate', line=dict(color='orange'), secondary_y=True,
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Estimate: %{y:.3f}<extra>Model</extra>'
    )

# Vol / variance style fields on primary axis
if 'backbone' in est_df.columns:
    bb_sigma = np.sqrt(est_df['backbone'].clip(lower=0))
    fig.add_scatter(
        x=est_df['dt'], y=bb_sigma, name='Backbone σ', line=dict(color='purple', dash='dot'), secondary_y=False,
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Backbone σ: %{y:.4f}<extra>Backbone σ</extra>'
    )

if 'seasonalSigma' in est_df.columns:
    fig.add_scatter(
        x=est_df['dt'], y=est_df['seasonalSigma'], name='Seasonal σ', line=dict(color='green', dash='dot'), secondary_y=False,
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Seasonal σ: %{y:.4f}<extra>Seasonal σ</extra>'
    )

if 'sigmaEwmShrunkNormalized' in est_df.columns:
    fig.add_scatter(
        x=est_df['dt'], y=est_df['sigmaEwmShrunkNormalized'], name='σ EWM Shrunk Norm', line=dict(color='teal', dash='dashdot'), secondary_y=False,
        hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>σ EWM Shrunk Norm: %{y:.4f}<extra>σ EWM Shrunk Norm</extra>'
    )

# Axis styling
fig.update_yaxes(title_text='BTC Price / σ', secondary_y=False)
fig.update_yaxes(title_text='Probability', range=[0,1], secondary_y=True)

# Center primary y-axis around target so its midpoint aligns with probability 0.5.
if targ is not None and 'btcPrice' in est_df.columns:
    price_series = est_df['btcPrice']
    max_dev = (price_series - targ).abs().max()
    if max_dev == 0:
        max_dev = max(abs(targ)*0.01, 1)
    pad = max_dev * 0.05
    fig.update_yaxes(range=[targ - max_dev - pad, targ + max_dev + pad], secondary_y=False)
else:
    if 'btcPrice' in est_df.columns:
        pmin, pmax = est_df['btcPrice'].min(), est_df['btcPrice'].max()
        pad = (pmax - pmin) * 0.05 if pmax > pmin else (pmax * 0.05 if pmax else 1)
        fig.update_yaxes(range=[pmin - pad, pmax + pad], secondary_y=False)

fig.update_layout(height=650, title='Trade vs Estimator', hovermode='x unified')
fig.show(renderer='browser')

In [ ]:
trades2 = df.copy()[['dt', 'side', 'outcome', 'name', 'price', 'size', 'proxyWallet']]

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

UP_LABELS = {"Yes", "YES", "Up", "UP"}

# up_price: normalize to UP outcome probability
trades2['up_price'] = trades2.apply(
    lambda r: float(r['price']) if r.get('outcome') in UP_LABELS else 1.0 - float(r['price']),
    axis=1
)

# up_side: perspective of the UP outcome (BUY means increasing UP exposure)
def _up_side(r):
    side = r.get('side')
    if r.get('outcome') in UP_LABELS:
        return side  # already oriented to UP
    # Outcome is DOWN: a BUY on DOWN reduces UP exposure => treat as SELL (and vice‑versa)
    if side == 'BUY':
        return 'SELL'
    if side == 'SELL':
        return 'BUY'
    return side

trades2['up_side'] = trades2.apply(_up_side, axis=1)

trades2[['dt', 'up_side', 'up_price', 'size', 'name', 'proxyWallet', 'side', 'outcome']].tail(10)

,dt,up_side,up_price,size,name,proxyWallet,side,outcome
9,2025-08-23 13:13:13+00:00,BUY,0.001,599.98,0x86674715A977c452E3Af11A1ac99ED5d17019356-1750778348148,0x86674715a977c452e3af11a1ac99ed5d17019356,SELL,Down
8,2025-08-23 13:15:17+00:00,BUY,0.001,68.12,"sc,mnbszcnm,bzsdm,fc",0x577346971f8d0a1338bc340a7f540a0b6f1ddc71,SELL,Down
7,2025-08-23 13:17:47+00:00,BUY,0.001,5.05,Jaijai,0x49a2062020deb19eb15608072d3484a05b833103,SELL,Down
6,2025-08-23 13:20:09+00:00,BUY,0.001,230.00,TechNinjaX,0x959567abebfe0fb7c4fd4be0136b17d0fe391b16,SELL,Down
5,2025-08-23 13:20:49+00:00,BUY,0.001,1000.00,1749177280,0x104a733d68fa06e916daa5ba7ab9f464ace2d487,SELL,Down
4,2025-08-23 13:27:01+00:00,BUY,0.001,1065.76,8787dada,0x2edb696cea63b3d02f6400857403d38ce2f43780,SELL,Down
3,2025-08-23 13:30:07+00:00,BUY,0.001,8.00,,0x4b43d2a40699a9a48bfadef6d85f3440817a65ab,SELL,Down
2,2025-08-23 13:30:09+00:00,BUY,0.001,20.00,,0x51a02132af17252e6993c3b668ea462d46234a02,SELL,Down
1,2025-08-23 13:53:11+00:00,BUY,0.001,379.00,bluehero,0x4c66e891ae4bb54cd83b0849cd7e8bfee97ac7b4,SELL,Down
0,2025-08-23 14:02:19+00:00,BUY,0.001,5.00,ashonetop,0xf73678e43f00a1c1df6774c4160b9a08859563ef,SELL,Down
